# BIP-0322 Signature

Initially just trying to get a notebook implementation that passes the test vector provided


**NOTE: Currently unable to get the test vector to pass!**

## Tagged Hash of Message to Sign

In [1]:
from buidl.hash import tagged_hash

In [2]:
message = b"Hello World"
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

### Test Vector

https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-hashing 

In [3]:
test_vector = 'f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a'
assert(test_vector == tagged_hash.hex())

## Generate scriptPubKey

### Use Test Vector

https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-signing

- private key `L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k` [(Compressed Wallet Import Format)](https://komodoplatform.com/en/academy/bitcoin-private-key/#common-bitcoin-private-key-formats)
- corresponding address `bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l` (A Segwit address)


In [4]:
compressed_wif_private_key = 'L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k'

In [5]:
address_test_vector = 'bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l'

In [6]:
from buidl.ecc import PrivateKey
# Parse WIF
private_key = PrivateKey.parse(compressed_wif_private_key)
# Generate a pay-to-witness-public-key-hash address
p2wpkh_address = private_key.point.p2wpkh_address(network="mainnet")
print("Segwit Adress:", p2wpkh_address)
# Check test vector
assert(p2wpkh_address == address_test_vector)

Segwit Adress: bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l


## Create virtual to_spend transaction

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

In [7]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.script import Script,P2WPKHScriptPubKey
from buidl.helper import big_endian_to_int

### Create the single transaction input

Note: the script commands `[0x00, 0x20, message_hash]`
- `0` is OP_0, which pushes a 0 onto the stack
- `32` is PUSH32 which pushes the next 32 bytes onto the stack
- `message_hash` is that next 32 bytes. Which is a tagged_hash of the message being signed.


In [13]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = big_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash

# Note this used to be: commands = [0, 32, message_hash] (as per BIP)
# Changed after looking at BTC core impl (see verification notebook)
# It appears the PUSH32 is implied and added by the size of the message added to the stack
commands = [0, message_hash]
scriptSig = Script(commands)
print(scriptSig.raw_serialize().hex())
# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a


In [14]:
tx_in

0000000000000000000000000000000000000000000000000000000000000000:4294967295

### Create the single transaction output

In [15]:
from buidl.script import address_to_script_pubkey

In [16]:
# Value of tx output
value = 0

script_pubkey = address_to_script_pubkey(p2wpkh_address)

print(script_pubkey)
tx_out = TxOut(value,script_pubkey)

OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 


In [17]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [18]:
virtual_to_spend_tx


tx: b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 

## Create virtual to_sign transaction

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```

### Create single transaction input

In [19]:
from buidl.witness import Witness

In [21]:
# Identify the virtual_to_spend tx hash. 
pretx_hash = virtual_to_spend_tx.hash()
# Identifies the index of the output from the virtual_to_spend_tx to be "spent"
prevout_index = 0
sequence = 0


tx_input = TxIn(pretx_hash,prevout_index,script_sig=None,sequence=sequence)

# Adding witness doesn't affect the serialization? Maybe this is intentional?
serialized_input = tx_input.serialize()


### Create single transaction output

In [22]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)
tx_output.serialize()
print(scriptPubKey)

OP_RETURN 


### Create unsigned virtual to_sign transaction

In [23]:
# create transaction on mainnet
version=0
tx_inputs = [tx_input]
tx_outputs = [tx_output]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_sign_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [24]:
virtual_to_sign_tx.id()

'88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf'

## Simple Signature is just the witness encoded

From the BIP-0322:

`A simple signature consists of a witness stack, consensus encoded as a vector of vectors of bytes, and base64-encoded. Validators should construct to_spend and to_sign as defined below, with default values for all fields except that`

## I think Signing Works Like this. Need to define a BIP-0322 way to resolve the prev_tx

In [25]:
# Have to manually set these, because cannot FETCH to_spend from network
# Note: Would ideally submit a P.R. to the buidl library to support this
virtual_to_sign_tx.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx.tx_ins[0]._value = 0
# print(virtual_to_sign_tx.tx_ins[0]._script_pubkey)
virtual_to_sign_tx.sign_input(0,private_key)



True

In [26]:
virtual_to_sign_tx.tx_ins[0].script_sig

In [27]:
virtual_to_sign_tx.verify_input(0)

True

## Serialize the Witness and Base64 Encode

In [34]:
# virtual_to_sign_tx.tx_ins[0].witness[0].serialize()
from buidl.helper import base64_encode
# 
bip322_signature = base64_encode(virtual_to_sign_tx.serialize_witness())
print("Copy across this signature output and use it in the verification notebook: \n")
print(bip322_signature)

Copy across this signature output and use it in the verification notebook: 

AkgwRQIhAOzyynlqt93lOKJr+wmmxIens//zPzl9tqIOua93wO6MAiBi5n5EyAcPScOjf1lAqIUIQtr3zKNeavYabHyR8eGhowEhAsfxIAMZZEKUPYWI4BruhAQjzFT8FSFSajuFwrDL1Yhy


## Signature Test Vector

Test vector taken from BIP0322

### NOTE: This currently does not pass

In [29]:
sig_test_vector = 'AkcwRAIgG3PASL/vRTgAqogWT6S8rUOQXNnfRzX6JncmbFlHc1ACIGQdsW+rnVmsQzyAYRQisHKFMigDmKiL7LUw4x17Fw5tASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='
assert(sig_test_vector == bip322_signature)

AssertionError: 

In [30]:
btc_core_sig_test_vectore = 'AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='
assert(btc_core_sig_test_vectore == bip322_signature)

AssertionError: 